<a href="https://colab.research.google.com/github/nyck33/tennisball-tracker/blob/master/tracknet_vs_tako.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TrackNet vs Tako

Tried Tracknet on fedNadal.mp4 and notice that it picks up some balls on players, lines and other non-balls.  Also, it loses the ball a lot on serves and other high speed shots so want to test it against Tako ball tracker that uses the blob detection and try to combine the two methods to not lose the ball.
Furthermore, try to use techniques in other papers to take away lines, players and audience areas.  
Finally, need to somehow detect bounces still.  

In [1]:
!pip install numpy
!pip install matplotlib
!pip install pillow
!pip install keras
!pip install opencv-python
!pip install pydot
!pip install h5py
!apt-get install graphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:

%cd /content/drive/"My Drive"/TrackNet-master/Code/TrackNet_Three_Frames_Input 
!pwd

/content/drive/My Drive/TrackNet-master/Code/TrackNet_Three_Frames_Input
/content/drive/My Drive/TrackNet-master/Code/TrackNet_Three_Frames_Input


In [6]:
!python predict_video.py  --save_weights_path=weights/model.3 --input_video_path="fedNadal.mp4" --output_video_path="fedNadal_TrackNet.mp4" --n_classes=256 

Using TensorFlow backend.
W0625 11:35:48.574276 139799186335616 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0625 11:35:48.586265 139799186335616 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0625 11:35:48.588393 139799186335616 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0625 11:35:48.600610 139799186335616 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0625 

# Tako Ball Tracker 

In [0]:
#tennis ball tracker from https://qiita.com/otakoma/items/04216c60fa31eae60947
#right now overdetects lines, nets, the line for the first row of the audience, net, player, other people,

import cv2
import sys
import numpy as np

#dilate useful in joining broken parts of an object after erasing noise with erosion
def dilation(dilationSize, kernelSize, img):  # 膨張した画像にして返す, return as an expanded image
    kernel = np.ones((kernelSize, kernelSize), np.uint8)
    #get a rectangulr kernel for use in dilation
    element = cv2.getStructuringElement( 
        cv2.MORPH_RECT, (2 * dilationSize + 1, 2 * dilationSize + 1), (dilationSize, dilationSize))
    #dilate: if all b, get b, else w, ie. bleed white into non-white areas
    dilation_img = cv2.dilate(img, kernel, element)
    #print(img, dilation_img)
    return dilation_img

#find ball in blob, returns a dilated img and array of ball pos
#how is it discerning ball from players and other objects?
def detect(gray_diff, thresh_diff=30, dilationSize=9, kernelSize=20):  # 一定面積以上の物体を検出
    #change all pixels in grayscale image to white or black based on threshold, ie. dark grey is black, light grey is 
    #white 
    retval, black_diff = cv2.threshold(
        gray_diff, thresh_diff, 255, cv2.THRESH_BINARY)  # 2値化
    #bleed white into non-white
    dilation_img = dilation(dilationSize, kernelSize, black_diff)  # 膨張処理 dilation, dilates ball
    #print("28 dilation_img {}".format(dilation_img))
    img = dilation_img.copy()
    #findContours finds, boundaries of image, 
    #outputs modified image, contours and hierarchy(each contour is array of coordinates of boundary pts)
    #RETR_EXTERNAL gets the outer most contour of the shape
    #CHAIN_APPROX_SIMPLE compresses contour to save memory, ie. 4 only saves 4 corners of rectangle
    contours, hierarchy = cv2.findContours(
        dilation_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # 境界線検出 
    #print("36 findContours: {} \n {} \n {}".format(image, contours, hierarchy))
    #array of ball pos, x, y 
    ball_pos = []

    for i in range(len(contours)):  # 重心位置を計算, calculate the center of the object ball
        count = len(contours[i])
        area = cv2.contourArea(contours[i])  # 面積計算
        #print("43 area".format(area))
        x, y = 0.0, 0.0
        for j in range(count):
            x += contours[i][j][0][0]
            y += contours[i][j][0][1]

        x /= count
        y /= count
        x = int(x)
        y = int(y)
        ball_pos.append([x, y]) #adding the x,y of contour of the ball?

    return ball_pos, img

#show the circle bounding box
def displayCircle(image, ballList, thickness=5):
    for i in range(len(ballList)):
        x = int(ballList[i][0])
        y = int(ballList[i][1])
        cv2.circle(image, (x, y), 10, (0, 0, 255), thickness)
        #print("63 image {}".format(image))
    return image

#resize images
def resizeImage(image, w=2, h=2):
    height = image.shape[0]
    width = image.shape[1]
    resizedImage = cv2.resize(image, (int(width / w), int(height / h)))
    return resizedImage

#resizing blob image to same as color image to concat
#Make the binarized image the same as the color array size
def blackToColor(bImage):
    #print("76 bImage {}".format(bImage))
    colorImage = np.array((bImage, bImage, bImage))
    colorImage = colorImage.transpose(1, 2, 0)
    #print("79 colorImage {}".format(colorImage))
    return colorImage


def run(input_video_path, output_video_path):
    video = cv2.VideoCapture(input_video_path)  # read video file
    #fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    if not video.isOpened():  # ファイルがオープンできない場合の処理.
        print("Could not open video")
        sys.exit()

    vidw = video.get(cv2.CAP_PROP_FRAME_WIDTH)
    vidh = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, #fps
                          (int(vidw), int(vidh)))  # 出力先のファイルを開く

    ok, frame = video.read()  # 最初のフレームを読み込む
    if not ok:
        print('Cannot read video file')
        sys.exit()
    #resize * 2##############################
    height, width, layers = frame.shape
    new_h = height * 2
    new_w = width * 2
    frame = cv2.resize(frame, (new_w, new_h))

    frame_pre = frame.copy()

    while True:
        ok, frame = video.read()  # フレームを読み込む
        if not ok:
            break
        #resize * 2##############################
        height, width, layers = frame.shape
        new_h = height * 2
        new_w = width * 2
        frame = cv2.resize(frame, (new_w, new_h))
        frame_next = frame.copy()

        #difference frames
        color_diff = cv2.absdiff(frame_next, frame_pre)  # フレーム間の差分計算
        #print("color_diff {}".format(color_diff))
        gray_diff = cv2.cvtColor(color_diff, cv2.COLOR_BGR2GRAY)  # グレースケール変換
        #print("gray_diff {}".format(gray_diff))

        #turn the grayscale diff frame into b and w
        retval, black_diff = cv2.threshold(
            gray_diff, 30, 255, cv2.THRESH_BINARY)#ThreshBinary is black if higher, else white 
        #print("black_diff {}".format(black_diff))
        #detect ball
        ball, dilation_img = detect(gray_diff)
        #print("ball {} \n\n dilation_img {}".format(ball, dilation_img))
        #takes one frame and puts circles around ball
        frame = displayCircle(frame, ball, 2)  # 丸で加工
        #print("frame with circle {}".format(frame))

        #Make the binarized image the same as the color array size
        cImage = blackToColor(dilation_img)  # 2値化画像をカラーの配列サイズと同じにする
        #print("141 dilation_img {} \n after: cImage {}".format(dilation_img, cImage))
        im1 = resizeImage(frame, 2, 2) 
        im2 = resizeImage(cImage, 2, 2)
        im_h = cv2.hconcat([im1, im2])  # 画像を横方向に連結

        #cv2.imshow("Tracking", im_h)  # フレームを画面表示
        out.write(im_h)

        frame_pre = frame_next.copy()  # 次のフレームの読み込み

        k = cv2.waitKey(1) & 0xff  # ESCを押したら中止
        if k == 27:
            break

    video.release()
    out.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    inputFile="fedNadal.mp4"
    outputFile="fedNadal-tako.avi"
    run(inputFile, outputFile)
